In [85]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### **MCQ Generator using Mistral AI API**

##### **0. Resources**

[iNeuron intellegence youtube video](https://www.youtube.com/watch?v=bsfobtZJCik&list=PLmQAMKHKeLZ-iTT-E2kK9uePrJ1Xua9VL&index=7&t=3s)

[AI Anytime youtube video](https://www.youtube.com/watch?v=Hcqmhhx30Pg&t=383s)

##### **1. Install Dependecies**

In [86]:
!pip install langchain
!pip install langchain_mistralai
!pip install PyPDF2
!pip install wikipedia

In [87]:
import os
import traceback
import pandas as pd
import json

from langchain_mistralai import ChatMistralAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2
import wikipedia

##### **API key Setup**

In [88]:
os.environ["MISTRAL_API_KEY"] = "ZL1Cxn38UvSt3Uk5cneYzAA4ba8HqkAh"

In [89]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    # model = "Mistral-7B-Instruct-v0.3",
    temperature=0.3,
    api_key = os.environ["MISTRAL_API_KEY"]
)

In [90]:
llm

ChatMistralAI(client=<httpx.Client object at 0x7ec9955800d0>, async_client=<httpx.AsyncClient object at 0x7ec995580280>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', model='mistral-large-latest', temperature=0.3)

##### **Template Creation**

In [91]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [92]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [93]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    output_key=["quiz"],
    template = TEMPLATE
)
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n')

In [94]:
TEMPLATE = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [95]:
quiz_evaluator_prompt = PromptTemplate(
    input_variables = ["quiz","subject"],
    output_variables=["review"],
    template = TEMPLATE
)
quiz_evaluator_prompt

PromptTemplate(input_variables=['quiz', 'subject'], input_types={}, partial_variables={}, template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. \nif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\nQuiz_MCQs:\n{quiz}\n\nCheck from an expert English Writer of the above quiz:\n')

##### **SequentialChain Creation**

In [96]:
quiz_generation_chain = LLMChain(llm=llm,
                                 prompt=quiz_generation_prompt,
                                 output_key="quiz",
                                 verbose=True)

In [97]:
quiz_evaluator_chain = LLMChain(llm=llm,
                                prompt=quiz_evaluator_prompt,
                                output_key="review",
                                verbose=True)

In [98]:
seq_quiz_chain = SequentialChain(chains=[quiz_generation_chain,quiz_evaluator_chain],
                                 input_variables=["text","number","subject","tone","response_json"],
                                 output_variables=["quiz","review"],
                                 verbose=True)

##### **Text Handling**

In [99]:
def extract_wikipedia_summary_to_file(topic, word_limit=500, filename="wikipedia_summary.txt"):
    try:
        wikipedia.set_lang("en")
        summary = wikipedia.summary(topic, auto_suggest=False)
        words = summary.split()
        limited_summary = ' '.join(words[:word_limit])

        if len(words) > word_limit:
            limited_summary += "..."

        with open(filename, 'w') as file:
            file.write(limited_summary)

        return f"Summary of '{topic}' has been saved to '{filename}'."

    except wikipedia.exceptions.PageError:
        return f"Sorry, the topic '{topic}' does not exist on Wikipedia."

    except wikipedia.exceptions.DisambiguationError as e:
        return f"Topic '{topic}' is ambiguous. Possible options include: {e.options}"

extract_wikipedia_summary_to_file("Machine Learning", word_limit=500, filename="Machine_Learning.txt")


"Summary of 'Machine Learning' has been saved to 'Machine_Learning.txt'."

##### **Result**

In [100]:
text = " "
with open("/content/Machine_Learning.txt","r") as f:
  text = f.readline()

In [101]:
text

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance. ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics. Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning. From a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing machine learning.'

In [102]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [104]:
response = seq_quiz_chain.invoke(input={"text":text,
                             "number":5,
                             "subject":"Data Science",
                             "tone":"Simple",
                             "response_json":json.dumps(RESPONSE_JSON)})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance. ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics. Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning. From a theoretical viewpoint, probab

In [122]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance. ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics. Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning. From a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing machine learning.',
 '

In [133]:
quiz_string = response.get("quiz")
cleaned_quiz_string = quiz_string.replace('```json\n', '').replace('```', '')
cleaned_quiz_json = json.loads(cleaned_quiz_string)

##### **Dataset Creation**

In [138]:
quiz_table_data = []
for key,value in cleaned_quiz_json.items():
  mcq = value["mcq"]
  options = " | ".join([f"{option_key} : {option_value}" for option_key,option_value in value["options"].items()])
  correct = value["correct"]

  quiz_table_data.append({"MCQ":mcq,"Options":options,"Corerct":correct})
quiz_table_data

[{'MCQ': 'What is machine learning primarily concerned with?',
  'Options': 'a : Developing robots | b : Studying statistical algorithms that can learn from data | c : Creating software applications | d : Designing computer hardware',
  'Corerct': 'b'},
 {'MCQ': 'Which field of study is related to machine learning and focuses on exploratory data analysis (EDA)?',
  'Options': 'a : Deep learning | b : Data mining | c : Natural language processing | d : Computer vision',
  'Corerct': 'b'},
 {'MCQ': 'What is the application of machine learning to business problems known as?',
  'Options': 'a : Data mining | b : Predictive analytics | c : Artificial intelligence | d : Neural networks',
  'Corerct': 'b'},
 {'MCQ': 'Which framework provides a theoretical viewpoint for describing machine learning?',
  'Options': 'a : PAC learning | b : Deep learning | c : Reinforcement learning | d : Supervised learning',
  'Corerct': 'a'},
 {'MCQ': 'In which decade did deep learning start making quick progre

In [141]:
quiz_df = pd.DataFrame(quiz_table_data)
quiz_df.to_csv("/content/Machine_Learning_quizes.csv")